# RAG Pipeline
사용자가 질문을 입력하면 시스템은 6단계 파이프라인을 통해 답변을 생성합니다. 먼저 `advanced_query_transformation()`으로 질문에 "movie" 키워드를 추가해 검색 정확도를 높이고, `advanced_query_routing()`으로 질문 유형에 따라 벡터 검색과 텍스트 검색 중 어떤 방법을 우선할지 결정합니다. 그 다음 `fusion_retrieval()`에서 ChromaDB의 의미 기반 검색과 BM25의 키워드 매칭을 동시에 수행해 10개 후보 문서를 추출하고, `rerank_documents()`가 BGE-M3 Reranker로 각 문서와 질문의 실제 관련성을 재평가해 순위를 재조정합니다. 상위 3개 문서는 `select_and_compress_context()`에서 GPT-4o가 2-3문장으로 요약해 토큰을 절약하고, 마지막으로 `generate_answer()`가 압축된 컨텍스트를 바탕으로 GPT-4o를 통해 자연스럽고 구체적인 최종 답변을 생성합니다.


```
사용자 질문
    ↓
[쿼리 변환] advanced_query_transformation() - 질문을 검색에 최적화된 형태로 변환 (movie 키워드 추가)
    ↓
[라우팅] advanced_query_routing() - 벡터 검색 vs 텍스트 검색 중 어떤 방법 우선할지 결정
    ↓
[퓨전 검색] fusion_retrieval() - ChromaDB(벡터) + BM25(키워드) 두 방법으로 동시 검색 → 10개 문서
    ↓
[리랭킹] rerank_documents() - BGE-M3 Reranker로 (질문, 문서) 쌍의 관련성 점수 계산 후 재정렬
    ↓
[압축] select_and_compress_context() - GPT-4o로 상위 3개 문서를 2-3문장으로 요약해서 컨텍스트 압축
    ↓
[생성] generate_answer() - GPT-4o가 압축된 컨텍스트 보고 최종 답변 작성
    ↓
답변 출력
```

# Installation of Dependencies


In [ ]:
!pip -q install transformers langchain_huggingface torch sentence-transformers chromadb langchain_openai rank-bm25 nltk pandas FlagEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!wget -q https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/05.%20Vector%20Similarity/dataset/movies_metadata.csv

#Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import chromadb
import os
from google.colab import userdata
import nltk
from rank_bm25 import BM25Okapi
from tqdm.auto import tqdm
from FlagEmbedding import FlagReranker

# Load Data

In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
df = pd.read_csv('movies_metadata.csv')
df = df[['original_title', 'overview']].dropna()
df = df.head(1000)  # 처리 속도를 위해 1000개만 사용

print(f"Loaded {len(df)} movies")
df.head()

/tmp/ipython-input-236619220.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('movies_metadata.csv')


Loaded 1000 movies


,original_title,overview
0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...


# Setup OpenAI API

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = "여러분이 발급받은 오픈AI 키 값"

def get_llm():
    """
    Returns the language model instance.

    Returns:
        ChatOpenAI: An instance of the ChatOpenAI language model.
    """
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0,
        max_tokens=1024,
    )
    return llm

In [ ]:
chat_openai_model = get_llm()

# Load the Embedding Model




SentenceTransformer의 all-MiniLM-L6-v2 모델을 로드하며, 문장을 벡터로 변환하는 임베딩 모델입니다.
사용자 질문과 영화 문서를 벡터로 변환해 ChromaDB에 저장하고, 나중에 코사인 유사도 기반 의미 검색에 사용됩니다.

In [ ]:
print("Loading sentence transformer model...")
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

Loading sentence transformer model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Load the reranking model

BGE Reranker v2-m3 모델을 로드합니다. 이 모델은 나중에 Fusion Retrieval로 검색된 문서들을 (질문, 문서) 쌍의 관련성 점수로 재평가해 진짜 관련 있는 문서를 상위로 올리는 리랭킹 용도로 사용됩니다.

In [ ]:
rerank_model = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True)

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

# Prepare Documents

DataFrame의 각 영화를 딕셔너리로 변환하며, 제목과 줄거리를 합쳐서 "content" 필드에 저장합니다.
예를 들어 "Toy Story: A cowboy doll is profoundly threatened..." 형태로 만들어 검색 시 제목과 줄거리 정보를 동시에 활용할 수 있도록 합니다.

In [ ]:
# Combine title and overview for better context
documents = []
for idx, row in df.iterrows():
    doc_text = f"{row['original_title']}: {row['overview']}"
    documents.append({
        "id": str(idx),
        "title": row['original_title'],
        "content": doc_text
    })

print(f"Prepared {len(documents)} documents")

Prepared 1000 documents


# Setup ChromaDB for Vector Search

ChromaDB에 "movies" 컬렉션을 생성하고 1000개 영화 문서를 384차원 벡터로 변환해 저장합니다.
기존에 같은 이름의 컬렉션이 있으면 먼저 삭제해서 중복 데이터 저장이나 충돌 에러를 방지합니다.
컬렉션 이름("movies")을 지정해두면 나중에 client.get_collection(name="movies")로 동일한 데이터를 다시 불러올 수 있고, 다른 데이터셋과 분리 관리할 수 있습니다.
저장된 벡터 DB는 이후 fusion_retrieval() 함수에서 collection.query()를 호출해 사용자 질문과 유사한 영화를 검색하는데 사용됩니다.

In [ ]:
# Initialize ChromaDB client and create collection
client = chromadb.Client()

# Define the collection name
collection_name = "movies"

try:
    # Delete existing collection if it exists
    try:
        client.delete_collection(name=collection_name)
    except:
        pass

    # Create new collection
    collection = client.create_collection(name=collection_name)
    print(f"Collection '{collection_name}' created successfully.")

    # Insert documents into the collection
    ids = [doc["id"] for doc in documents]
    contents = [doc["content"] for doc in documents]

    collection.add(ids=ids, documents=contents)
    print(f"Inserted {len(documents)} documents into ChromaDB.")

except Exception as e:
    print(f"An error occurred: {e}")

Collection 'movies' created successfully.


/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 35.3MiB/s]


Inserted 1000 documents into ChromaDB.


# Setup BM25 for Textual Search

1000개 영화 문서를 소문자로 변환 후 공백 기준으로 토큰화하여 BM25 인덱스를 생성합니다.
BM25는 통계 기반 키워드 검색 알고리즘으로, 질문 단어가 문서에 얼마나 등장하는지 계산해 관련성 점수를 매깁니다. 이 실습에서는 나중에 fusion_retrieval()에서 ChromaDB의 의미 검색과 함께 사용됩니다.

In [ ]:
# Tokenize documents for BM25
print("Setting up BM25...")
tokenized_corpus = [doc["content"].lower().split() for doc in documents]
bm25 = BM25Okapi(tokenized_corpus)
print("BM25 setup complete!")

Setting up BM25...
BM25 setup complete!


# Advanced Query Transformation


사용자 질문에 "movie"나 "film" 단어가 없으면 자동으로 " movie"를 뒤에 붙여서 영화 관련 검색 결과를 더 잘 찾도록 쿼리를 확장하는 함수입니다.

In [ ]:
def advanced_query_transformation(query):
    """
    Transforms the input query by adding synonyms, extensions, or modifying the structure
    for better search performance.

    Args:
        query (str): The original query.

    Returns:
        str: The transformed query with added synonyms or related terms.
    """
    # Simple expansion - in production you'd use more sophisticated methods
    expanded_query = query

    # Add common movie-related terms if not present
    movie_terms = ["movie", "film"]
    if not any(term in query.lower() for term in movie_terms):
        expanded_query = query + " movie"

    return expanded_query

# Advanced Query Routing

질문에 "title", "named", "called", "specific" 같은 키워드가 있으면 'textual'(키워드 검색)을 반환하고, 없으면 'vector'(의미 검색)를 반환하는 함수입니다. 검색 방법을 자동으로 선택해줍니다.

In [ ]:
def advanced_query_routing(query):
    """
    Determines the retrieval method based on the presence of specific keywords in the query.

    Args:
        query (str): The user's query.

    Returns:
        str: 'textual' if the query requires text-based retrieval, 'vector' otherwise.
    """
    # Keywords that suggest textual search is better
    textual_keywords = ["title", "named", "called", "specific"]

    if any(keyword in query.lower() for keyword in textual_keywords):
        return "textual"
    else:
        return "vector"

# Fusion Retrieval Function

질문을 벡터 검색(ChromaDB)과 키워드 검색(BM25) 두 방법으로 동시에 검색한 뒤 결과를 합치는 하이브리드 검색 함수입니다.
벡터 검색은 질문을 임베딩으로 변환해 의미가 유사한 영화를 찾고, BM25는 키워드 매칭으로 정확한 단어가 포함된 영화를 찾습니다.
두 검색 결과를 합친 뒤 중복을 제거하고, top_k*2(기본 10개)를 반환해서 다음 단계인 reranking에서 더 정확하게 재정렬할 수 있도록 합니다.

In [ ]:
def fusion_retrieval(query, top_k=5):
    """
    Retrieves the top_k most relevant documents using a combination of vector-based
    and textual retrieval methods.

    Args:
        query (str): The search query.
        top_k (int): The number of top documents to retrieve.

    Returns:
        list: A list of combined results from both vector and textual retrieval methods.
    """
    # Vector-based retrieval using sentence embeddings
    query_embedding = sentence_model.encode(query).tolist()
    vector_results = collection.query(
        query_embeddings=[query_embedding],
        n_results=min(top_k, len(documents))
    )

    # Textual retrieval using BM25
    tokenized_query = query.lower().split()
    bm25_scores = bm25.get_scores(tokenized_query)

    # Get top_k indices from BM25
    top_bm25_indices = np.argsort(bm25_scores)[-top_k:][::-1]
    bm25_documents = [documents[i]["content"] for i in top_bm25_indices]

    # Combine results from both retrieval methods
    vector_docs = vector_results['documents'][0] if vector_results['documents'] else []
    combined_results = vector_docs + bm25_documents

    # Remove duplicates while preserving order
    seen = set()
    unique_results = []
    for doc in combined_results:
        if doc not in seen:
            seen.add(doc)
            unique_results.append(doc)

    return unique_results[:top_k * 2]  # Return more docs for reranking

# Document Reranking Function

Fusion Retrieval로 검색된 10개 문서를 BGE Reranker 모델로 재평가하는 함수입니다.
(질문, 문서) 쌍을 만들어 배치로 한 번에 관련성 점수를 계산하고, 점수가 높은 순서대로 재정렬해서 반환합니다.
초기 검색에서는 놓쳤을 수 있는 진짜 관련성 높은 문서를 상위로 올려 최종 답변 품질을 높입니다.

In [ ]:
def rerank_documents(query, documents_list):
    if not documents_list:
        return []

    pairs = [[query, doc] for doc in documents_list]
    scores = rerank_model.compute_score(pairs, normalize=True)

    if not isinstance(scores, list):
        scores = [scores]

    ranked_docs = sorted(zip(documents_list, scores), key=lambda x: x[1], reverse=True)
    return [doc for doc, score in ranked_docs]

# Context Selection and Compression

재정렬된 문서 중 상위 3개를 GPT-4o로 요약하는 함수입니다.
각 영화 문서를 2-3문장으로 압축해 최종 답변 생성 시 전달할 컨텍스트 길이를 줄이고, 30단어 미만의 짧은 문서는 그대로 사용하며 요약 실패 시 원본 텍스트를 반환합니다.

In [ ]:
def select_and_compress_context(documents_list, max_docs=3):
    """
    Summarizes the content of the retrieved documents to create a compressed context.

    Args:
        documents_list (list): A list of documents to summarize.
        max_docs (int): Maximum number of documents to use.

    Returns:
        list: A list of summarized texts for each document.
    """
    summarized_context = []

    for doc in documents_list[:max_docs]:
        try:
            input_length = len(doc.split())

            # Skip if too short
            if input_length < 30:
                summarized_context.append(doc)
                continue

            # Use GPT-4o for summarization
            prompt = f"""Summarize the following movie description concisely in 2-3 sentences:

{doc}

Summary:"""

            response = chat_openai_model.invoke(prompt)
            summary = response.content

            summarized_context.append(summary)
        except Exception as e:
            # If summarization fails, use original text
            summarized_context.append(doc)

    return summarized_context

# Answer Generation Function

압축된 3개 영화 컨텍스트를 GPT-4o에게 전달해 최종 답변을 생성하는 함수입니다.
프롬프트에 역할(영화 추천 전문가), 사용자 질문, 검색된 영화 정보를 넣어 GPT-4o를 호출하고, 자연스러운 한국어/영어 답변을 반환합니다.
검색된 실제 영화 데이터를 기반으로 답변하기 때문에 GPT-4o만 사용하는 것보다 훨씬 정확하고 구체적인 추천이 가능합니다.

In [ ]:
def generate_answer(query, chunks, llm):
    """
    Generates an answer based on the input query and context chunks using a language model.

    Args:
        query (str): The user's query.
        chunks (list): A list of context chunks to inform the answer.
        llm (ChatOpenAI): An instance of the ChatOpenAI language model.

    Returns:
        str: The generated answer.
    """
    # Combine chunks into a single context string
    context = "\n\n".join(chunks)

    # Construct the prompt for the language model
    prompt = f"""You are an expert movie recommendation assistant. Based on the provided context about movies, answer the following question comprehensively.

Question: {query}

Context:
{context}

Please provide a detailed and helpful answer based on the context above. If recommending movies, explain why they match the query."""

    # Invoke the language model with the prompt
    response = llm.invoke(prompt)

    # Extract the content from the response
    generated_text = response.content

    return generated_text

# Full Advanced RAG Pipeline

위에서 정의한 모든 함수를 순차적으로 실행하는 메인 파이프라인 함수입니다.
사용자 질문을 입력받아 (1)쿼리 변환 → (2)라우팅 → (3)퓨전 검색 → (4)리랭킹 → (5)컨텍스트 압축 → (6)GPT-4o 답변 생성 순서로 처리하며, 각 단계마다 중간 결과를 출력해 전체 과정을 추적할 수 있습니다.

In [ ]:
def advanced_rag_pipeline(query):
    print(f"\n{'='*60}")
    print(f"Processing query: {query}")
    print(f"{'='*60}\n")

    # Transform and route query
    print("1. Transforming query...")
    transformed_query = advanced_query_transformation(query)
    print(f"   Original: {query}")
    print(f"   Transformed: {transformed_query}")

    print("\n2. Routing query...")
    retrieval_method = advanced_query_routing(transformed_query)
    print(f"   Retrieval method: {retrieval_method}")

    # Retrieve documents using fusion retrieval
    print("\n3. Retrieving documents (Fusion: Vector + BM25)...")
    retrieved_documents = fusion_retrieval(transformed_query, top_k=5)
    print(f"   Retrieved {len(retrieved_documents)} documents")
    print(f"   Retrieved documents:")
    for i, doc in enumerate(retrieved_documents, 1):
        print(f"   [{i}] {doc}")

    # Rerank documents based on relevance
    print("\n4. Reranking documents...")
    ranked_documents = rerank_documents(query, retrieved_documents)
    print(f"   Reranked {len(ranked_documents)} documents")
    print(f"   Reranked documents:")
    for i, doc in enumerate(ranked_documents, 1):
        print(f"   [{i}] {doc}")

    # Select and compress context for answer generation
    print("\n5. Compressing context...")
    context = select_and_compress_context(ranked_documents, max_docs=3)
    print(f"   Compressed to {len(context)} context chunks")
    print(f"   Compressed contexts:")
    for i, chunk in enumerate(context, 1):
        print(f"   [{i}] {chunk}")

    # Generate final answer based on the context
    print("\n6. Generating answer with GPT-4o...")
    final_answer = generate_answer(query, context, chat_openai_model)

    print(f"\n{'='*60}")
    print("Answer generated successfully!")
    print(f"{'='*60}\n")

    return final_answer

# Example Usage

3개의 샘플 질문으로 RAG 파이프라인을 테스트하는 코드입니다.
먼저 첫 번째 질문("What are some good movies to watch on a rainy day?")만 실행해 결과를 확인하고, 이어서 3개 질문 모두를 반복문으로 실행해 각각의 답변을 출력합니다.

In [ ]:
# Example queries
queries = [
    "What are some good movies to watch on a rainy day?",
    "Recommend me action movies with great special effects",
    "What movies are similar to The Shawshank Redemption?"
]

# Run the first query through the Advanced RAG Pipeline
query = queries[0]
answer = advanced_rag_pipeline(query)

# Output the generated answer
print("\n" + "="*60)
print("FINAL ANSWER:")
print("="*60)
print(answer)
print("="*60)

"""# Test Multiple Queries"""

# Test all queries
print("\n\n" + "🎬 TESTING MULTIPLE QUERIES 🎬".center(60, "="))

for i, query in enumerate(queries, 1):
    print(f"\n\n{'='*60}")
    print(f"QUERY {i}: {query}")
    print(f"{'='*60}")

    answer = advanced_rag_pipeline(query)

    print("\n📝 ANSWER:")
    print("-" * 60)
    print(answer)
    print("="*60)


Processing query: What are some good movies to watch on a rainy day?

1. Transforming query...
   Original: What are some good movies to watch on a rainy day?
   Transformed: What are some good movies to watch on a rainy day?

2. Routing query...
   Retrieval method: vector

3. Retrieving documents (Fusion: Vector + BM25)...
   Retrieved 10 documents
   Retrieved documents:
   [1] Singin' in the Rain: In 1927 Hollywood, Don Lockwood and Lina Lamont are a famous on-screen romantic pair in silent movies, but Lina mistakes the on-screen romance for real love. When their latest film is transformed into a musical, Don has the perfect voice for the songs, but strident voice faces the studio to dub her voice. Aspiring actress, Kathy Selden is brought in and, while she is working on the movie, Don falls in love with her.
   [2] Before the Rain: The circularity of violence seen in a story that circles on itself. In Macedonia, during war in Bosnia, Christians hunt an ethnic Albanian girl who ma